In [2]:
print __doc__
from time import time
import logging
import pylab as pl
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.datasets import fetch_lfw_people
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import RandomizedPCA
from sklearn.svm import SVC

# Display progress logs on stdout                                                                                                                                                         
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

###############################################################################                                                                                                           
# Download the data, if not already on disk and load it as numpy arrays                                                                                                                   
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

Automatically created module for IPython interactive environment


/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [28]:
# introspect the images arrays to find the shapes (for plotting)                                                                                                                          
n_samples, h, w = lfw_people.images.shape
np.random.seed(42)
print "h= ",h
print "w= ",w
print 'lfw_people.images.shape is ', type(lfw_people.images.shape)
print "lfw_people.images is ",type(lfw_people.images)
print 'lfw_people.data is ',type(lfw_people.data)
# for machine learning we use the data directly (as relative pixel                                                                                                                        
# position info is ignored by this model)                                                                                                                                                 
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person                                                                                                                                            
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print "lfw_people.targe is ",type(lfw_people.target)

print "Total dataset size:"
print "n_samples: %d" % n_samples
print "n_features: %d" % n_features
print "n_classes: %d" % n_classes

h=  50
w=  37
lfw_people.images.shape is  <type 'tuple'>
lfw_people.images is  <type 'numpy.ndarray'>
lfw_people.data is  <type 'numpy.ndarray'>
lfw_people.targe is  <type 'numpy.ndarray'>
Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7


In [22]:
import numpy
a = numpy.array([[1,2,3],[4,5,6]])
print a.shape[1]
a[:,0]

3


array([1, 4])

In [24]:
# Split into a training and testing set                                                                                                                                                   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)



In [30]:
# Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled                                                                                                                    
# dataset): unsupervised feature extraction / dimensionality reduction                                                                                                                    
n_components = 150
print "Extracting the top %d eigenfaces from %d faces" % (n_components, X_train.shape[0])
t0 = time()
pca = RandomizedPCA(n_components=n_components, whiten=True).fit(X_train)
print "done in %0.3fs" % (time() - t0)

eigenfaces = pca.components_.reshape((n_components, h, w))
print "number of eigenfaces: ", eigenfaces.shape[0]
print "Projecting the input data on the eigenfaces orthonormal basis"
t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print "done in %0.3fs" % (time() - t0)

 Extracting the top 150 eigenfaces from 966 faces
done in 0.324s
number of eigenfaces:  150
Projecting the input data on the eigenfaces orthonormal basis
done in 0.038s


In [31]:
# Train a SVM classification model                                                                                                                                                        

print "Fitting the classifier to the training set"
t0 = time()
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }
# for sklearn version 0.16 or prior, the class_weight parameter value is 'auto'                                                                                                           
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(X_train_pca, y_train)
print "done in %0.3fs" % (time() - t0)
print "Best estimator found by grid search:"
print clf.best_estimator_

Fitting the classifier to the training set
done in 18.961s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [32]:
# Quantitative evaluation of the model quality on the test set                                                                                                                            

print "Predicting the people names on the testing set"
t0 = time()
y_pred = clf.predict(X_test_pca)
print "done in %0.3fs" % (time() - t0)

print classification_report(y_test, y_pred, target_names=target_names)
print confusion_matrix(y_test, y_pred, labels=range(n_classes))

Predicting the people names on the testing set
done in 0.058s
                   precision    recall  f1-score   support

     Ariel Sharon       0.57      0.62      0.59        13
     Colin Powell       0.74      0.90      0.81        60
  Donald Rumsfeld       0.80      0.74      0.77        27
    George W Bush       0.90      0.88      0.89       146
Gerhard Schroeder       0.77      0.80      0.78        25
      Hugo Chavez       0.82      0.60      0.69        15
       Tony Blair       0.90      0.75      0.82        36

      avg / total       0.83      0.83      0.83       322

[[  8   2   2   1   0   0   0]
 [  1  54   0   4   0   1   0]
 [  3   0  20   3   0   0   1]
 [  1  10   2 128   4   0   1]
 [  0   2   0   2  20   1   0]
 [  0   2   0   1   2   9   1]
 [  1   3   1   4   0   0  27]]


In [33]:
###############################################################################
# Split into a training set and a test set using a stratified k fold

# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


def pcaTrainAndPredict(n_components):
    ###############################################################################
    # Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled
    # dataset): unsupervised feature extraction / dimensionality reduction
#     n_components = 150

    print "Extracting the top %d eigenfaces from %d faces" % (n_components, X_train.shape[0])
    t0 = time()
    pca = RandomizedPCA(n_components=n_components, whiten=True).fit(X_train)
    print "done in %0.3fs" % (time() - t0)

    eigenfaces = pca.components_.reshape((n_components, h, w))

    print "Projecting the input data on the eigenfaces orthonormal basis"
    t0 = time()
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    print "done in %0.3fs" % (time() - t0)


    ###############################################################################
    # Train a SVM classification model

    print "Fitting the classifier to the training set"
    t0 = time()
    param_grid = {
             'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
              }
    clf = GridSearchCV(SVC(kernel='rbf', class_weight='auto'), param_grid)
    clf = clf.fit(X_train_pca, y_train)
    print "done in %0.3fs" % (time() - t0)
    print "Best estimator found by grid search:"
    print clf.best_estimator_


    ###############################################################################
    # Quantitative evaluation of the model quality on the test set

    print "Predicting the people names on the testing set"
    t0 = time()
    y_pred = clf.predict(X_test_pca)
    print "done in %0.3fs" % (time() - t0)

    print classification_report(y_test, y_pred, target_names=target_names)
    print confusion_matrix(y_test, y_pred, labels=range(n_classes))


pcaTrainAndPredict(150)
###############################################################################
# Qualitative evaluation of the predictions using matplotlib

def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    pl.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    pl.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        pl.subplot(n_row, n_col, i + 1)
        pl.imshow(images[i].reshape((h, w)), cmap=pl.cm.gray)
        pl.title(titles[i], size=12)
        pl.xticks(())
        pl.yticks(())


# plot the result of the prediction on a portion of the test set

def title(y_pred, y_test, target_names, i):
    pred_name = target_names[y_pred[i]].rsplit(' ', 1)[-1]
    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)

prediction_titles = [title(y_pred, y_test, target_names, i)
                         for i in range(y_pred.shape[0])]

plot_gallery(X_test, prediction_titles, h, w)

# plot the gallery of the most significative eigenfaces

eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
plot_gallery(eigenfaces, eigenface_titles, h, w)

pl.show()

Extracting the top 150 eigenfaces from 966 faces
done in 0.304s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.037s
Fitting the classifier to the training set


/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated

done in 19.928s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='auto', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Predicting the people names on the testing set
done in 0.054s
                   precision    recall  f1-score   support

     Ariel Sharon       0.61      0.85      0.71        13
     Colin Powell       0.77      0.83      0.80        60
  Donald Rumsfeld       0.68      0.70      0.69        27
    George W Bush       0.92      0.88      0.90       146
Gerhard Schroeder       0.80      0.80      0.80        25
      Hugo Chavez       0.69      0.60      0.64        15
       Tony Blair       0.88      0.83      0.86        36

      avg / total       0.84      0.83      0.83       322

[[ 11   0   2   0   0   0   0]
 [  1  50   3   4   0   1   1]
 [  4   0  19   2   0   1   1]
 [  2   9   2 128   3   1   1]

In [34]:
list_n_components= [10, 15, 25, 50, 100, 250]
for n in list_n_components:
    pcaTrainAndPredict(n)

Extracting the top 10 eigenfaces from 966 faces
done in 0.125s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.027s
Fitting the classifier to the training set


/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated

done in 51.824s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='auto', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Predicting the people names on the testing set
done in 0.011s
                   precision    recall  f1-score   support

     Ariel Sharon       0.11      0.15      0.12        13
     Colin Powell       0.42      0.53      0.47        60
  Donald Rumsfeld       0.26      0.33      0.30        27
    George W Bush       0.66      0.60      0.63       146
Gerhard Schroeder       0.14      0.16      0.15        25
      Hugo Chavez       0.40      0.13      0.20        15
       Tony Blair       0.41      0.33      0.37        36

      avg / total       0.48      0.46      0.46       322

[[ 2  6  1  3  1  0  0]
 [ 7 32  2  9  5  1  4]
 [ 2  8  9  6  1  0  1]
 [ 6 21 10 87 12  1  9]
 [ 0  3  4 10  4  1  3]
 [ 1 

/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated

done in 15.157s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='auto', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Predicting the people names on the testing set
done in 0.012s
                   precision    recall  f1-score   support

     Ariel Sharon       0.45      0.38      0.42        13
     Colin Powell       0.66      0.73      0.69        60
  Donald Rumsfeld       0.47      0.52      0.49        27
    George W Bush       0.74      0.75      0.75       146
Gerhard Schroeder       0.65      0.44      0.52        25
      Hugo Chavez       0.54      0.47      0.50        15
       Tony Blair       0.56      0.56      0.56        36

      avg / total       0.65      0.66      0.65       322

[[  5   3   1   4   0   0   0]
 [  5  44   1   9   1   0   0]
 [  0   4  14   9   0   0   0]
 [  1  11  13 110   2   2   7]
 

...

done in 6.662s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='auto', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Predicting the people names on the testing set
done in 0.021s
                   precision    recall  f1-score   support

     Ariel Sharon       0.64      0.69      0.67        13
     Colin Powell       0.82      0.90      0.86        60
  Donald Rumsfeld       0.65      0.56      0.60        27
    George W Bush       0.85      0.89      0.87       146
Gerhard Schroeder       0.76      0.64      0.70        25
      Hugo Chavez       0.77      0.67      0.71        15
       Tony Blair       0.78      0.69      0.74        36

      avg / total       0.80      0.80      0.80       322

[[  9   2   1   1   0   0   0]
 [  0  54   1   4   1   0   0]
 [  3   2  15   7   0   0   0]
 [  0   5   5 130   1   1   4]
 

/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated

done in 12.329s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='auto', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.005, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Predicting the people names on the testing set
done in 0.041s
                   precision    recall  f1-score   support

     Ariel Sharon       0.67      0.77      0.71        13
     Colin Powell       0.83      0.90      0.86        60
  Donald Rumsfeld       0.85      0.63      0.72        27
    George W Bush       0.90      0.95      0.92       146
Gerhard Schroeder       0.78      0.84      0.81        25
      Hugo Chavez       0.90      0.60      0.72        15
       Tony Blair       0.90      0.75      0.82        36

      avg / total       0.86      0.86      0.86       322

[[ 10   0   2   1   0   0   0]
 [  1  54   0   3   1   1   0]
 [  2   2  17   6   0   0   0]
 [  1   5   0 139   0   0   1]

/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/yuchengtsai/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated

done in 30.927s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='auto', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Predicting the people names on the testing set
done in 0.087s
                   precision    recall  f1-score   support

     Ariel Sharon       0.65      0.85      0.73        13
     Colin Powell       0.79      0.93      0.85        60
  Donald Rumsfeld       0.86      0.70      0.78        27
    George W Bush       0.93      0.93      0.93       146
Gerhard Schroeder       0.87      0.80      0.83        25
      Hugo Chavez       0.80      0.53      0.64        15
       Tony Blair       0.82      0.75      0.78        36

      avg / total       0.86      0.86      0.86       322

[[ 11   0   2   0   0   0   0]
 [  1  56   1   1   0   1   0]
 [  3   2  19   1   0   0   2]
 [  1   7   0 136   0   1   1]

In [37]:
1.-4./29.

0.8620689655172413

In [39]:
import numpy
for i in range(3):
    for j in range(4):
        print numpy.array([i,j]) 

[0 0]
[0 1]
[0 2]
[0 3]
[1 0]
[1 1]
[1 2]
[1 3]
[2 0]
[2 1]
[2 2]
[2 3]


In [51]:
for i in range(3):
    i = j
    print i,j

2 2
2 2
2 2
